### Exercise: Differential Gene Expression Analysis
In this exercise, we will calculate and analyse differentially expressed genes between two different conditions of in vitro stimulated PBMCs.

#### Import required packages and data
Following sc-best-practices (https://www.sc-best-practices.org/conditions/differential_gene_expression.html), we will use the Kang dataset, which is a 10x droplet-based scRNA-seq peripheral blood mononuclear cell (PBMC) data from 8 Lupus patients before and after 6h-treatment with INF-β (16 samples in total) (Kang et al., 2018).

You need to download the dataset from the course github repository (https://github.com/buchauer-lab/charite-sc-data-course/tree/main/materials/Day4), unzip it, and use the correct path to the data on your system in the import function below.

In [ ]:
# general data handling
import numpy as np
import pandas as pd
from scipy import sparse

# single cell analysis
import scanpy as sc
import decoupler as dc

# differential expression testing
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# adjust the path to the location of the folder "Kang2018" on your system
kang_data_path = 

In [ ]:
# import single cell data using sc.read_10x_mtx
adata = 

In [ ]:
# inspect


In [ ]:
# import meta data
metadata_df = 

In [ ]:
# inspect the metadata


In [ ]:
# add metadata to adata.obs using the .join method


In [ ]:
# inspect adata.obs to ensure everything worked


In [ ]:
# print the different cell type labels which appear here to screen
adata.obs['cell_type'].unique()

In [ ]:
# print the patient identifiers


In [ ]:
# print the different conditions, here called 'label'


#### Data preparation
We will perform a very basic quality control and calculate pseudobulks for each patient which we will need for differential expression analysis with pyDESeq2.


In [ ]:
# use scanpy's filter_cells and filter_genes functions to remove cells with 
# less than 200 genes and genes which appear in less than 3 cells


In [ ]:
# next, we need to introduce a column that holds the sample information, in our case a combination of "replicate" and "label"
adata.obs['sample'] = adata.obs['replicate'] + '_' + adata.obs['label']

In order to create pseudobulks (gene expression vectors per sample, generated by summing up all single cell expression vectos of a given cell type in that sample), we will use functionality from `decopupler`.

In [ ]:
# use the decoupler function dc.pp.pseudobulk to generate an anndata object of pseudobulks
# use mode='sum' to create pseudobulks by summing
pb_data = 

In [ ]:
# inspect the object - how many observations are in there now? Is this according to your expectation?


In [ ]:
# inspect the metadata in the obs dataframe


In [ ]:
# now, we want to remove pseudobulks which do not have at least 30 cells or 3000 counts
# use the function dc.pp.filter_samples to accomplish this


#### Basic analysis of the pseudobulked data objects
In differential expression testing, defining the right statistical model is very important. In order to identify the covariates which drive differences in our dataset (these are the covariates we should include in our model), we use scanpy to calculate a PCA for the pseudobulked data and inspect a series of PCA plots coloured by covariates.

In [ ]:
# we make a copy of the data object above as we will lognormalize it for calculating the PCA
pb_copy = pb_data.copy()

In [ ]:
# the .obs metadata contains a column psbulk_counts with count sums, add a column which contains the np.log of this
pb_copy.obs['log_psbulk_counts'] = np.log(pb_copy.obs['psbulk_counts'])

In [ ]:
# as you would for single cell data, perform the following steps using scanpy functions:
# 1. normalize total counts to a target sum 1e6
# 2. log1p transform the data
# 3. calculate a PCA


In [ ]:
# plot PCA plots coloured by label, cell_type, replicate, log_lib_size


Which of the above covariates seem to be strongly driving variance in PC1  or PC2? Discuss.

#### Differential expression analysis with PyDESeq2
We will now use the pseudobulked data of monocytes only for differential expression testing. We first need to get the data into the right format - PyDESeq2 expects a pandas dataframe of the gene expression data as well as of the metadata.

In [ ]:
# First, subset the pseudobulked anndata object to only the monoyte populations.
mono_pb = pb_data[pb_data.obs['cell_type']=='CD14+ Monocytes']

In [ ]:
# Next, we want to remove genes for which we do not expect relevant results
# Discuss with your neighbor why we want to reduce the number of genes for which we 
# perform differential expression testing.
# we want to remove all genes for which we do not see at least 15 counts in total
# use dc.pp.filter_by_expr to accomplish this


# we want to retain only genes which are present in at least 10% of cells in at least 3 samples
# use dc.pp.filter_prop to accomplish this


How many genes did we lose through this filtering?

In [ ]:
# Let's make a dataframe of the counts
counts_df = pd.DataFrame(data=mono_pb.X, index=mono_pb.obs.index, columns=mono_pb.var.index)

In [ ]:
# Insepct, always inspect...


In [ ]:
# We also need the metadata in a corresponding format with sample as index
metadata_df = mono_pb.obs

In [ ]:
# Inspect!


#### Now for the actual differential expression testing!
The steps below follow the PyDESeq2 tutorial at https://pydeseq2.readthedocs.io/en/stable/auto_examples/plot_minimal_pydeseq2_pipeline.html#sphx-glr-auto-examples-plot-minimal-pydeseq2-pipeline-py which you can visit for further info.
The important part below is the definition of the design matrix, i.e. which covariate we expect the gene expression to depend on. In our case, this is the metadata entry "label" which represents the conditions control vs. stimulated.

In [ ]:
# First step - initialize a DeSeqDataSet, the object which holds our data and later on also the results
inference = DefaultInference(n_cpus=None)
dds = DeseqDataSet(
    counts=counts_df,
    metadata=metadata_df,
    design="~label",
    refit_cooks=True,
    inference=inference,
)

In [ ]:
# Second step - fitting dispersions, size factors etc for each gene
dds.deseq2()

In [ ]:
# Never hurts to inspect the object (actually, dds objects use AnnData as a backbone!)
dds

In [ ]:
# We can use anndata-like calls to inspect fitted dispersions and log fold changes:
print(dds.var["dispersions"])
print(dds.varm["LFC"])

In [ ]:
# Third step - no we use the fitted dispersions and LFCs to do actual statistical tests.
# For this, a new class is introduced: DeSeqStats objects.
ds = DeseqStats(dds, contrast=["label", "ctrl", "stim"], inference=inference)

In [ ]:
# using this object, a Wald test can be run and results displayed like this:
ds.summary()

### Exploring and plotting results
Results of the tests are available in tabular form in `ds.results_df`. 

In [ ]:
# for ease of access, we will first reassign the result dataframe
results_df = ds.results_df

In [ ]:
# inspect


#### 1. Compare top genes by different criteria

In [ ]:
# sort the dataframe by adj p val using .sort_values method and inspect the top 10 genes


In [ ]:
# sort the dataframe by highest fold change and inspect the top 10 genes


In [ ]:
# sort the dataframe by lowest fold change and inspect the top 10 genes


Why do these lists differ? Do some genes appear in several lists?

#### 2. Smear plot and volcano plot

In [ ]:
# Create smear plot - a 2D scatter plot with log2FoldChange on the y axis and expression on the x Axis
# use plt.scatter to achieve this
# plot np.log2(results_df['baseMean'] + 1) on the x axis
# highlight significant genes, i.e. genes with pdj < 0.5 in red
# (a simple way to achieve this is to just plot the relevant genes again in red)


Describe the plot, what is the general behavior we observe for significant results?

In [ ]:
# use the function dc.pl.volcano to plot a volcano plot
# show the names of the top 10 differentially expressed features
# set the significance threshold to 1 log2FoldChange
# save the figure to file


#### 3. Heatmap of top diferentially expressed genes

In [ ]:
# Find the 10 top genes by FC in the stim group and the 10 top genes by FC in the control group
# and combine them into a list
l1 = results_df.sort_values('log2FoldChange', ascending=True).head(10).index.tolist()
l2 = results_df.sort_values('log2FoldChange', ascending=False).head(10).index.tolist()
top_genes = l1+l2

In [ ]:
# We want to plot normalized counts which are stored in the dds object.
# For better visibility, we add a log version of these counts
dds.layers["log_normed_counts"] = np.log(dds.layers["normed_counts"]+1)

In [ ]:
# Plot a heatmap of normalized counts for the top_genes list as a heatmap
# Use sc.pl.heatmap on the dds object to accomplish this. use the layer log_normed_counts


#### 4. Individual gene expression plots

In [ ]:
# pick 2 or 3 genes and plot their expression by condition in box plots, use sc.pl.violin for this


#### 5. Gene annotation lookup
Look up some of your top genes to find out what pathways they are involved in and what their function is. Here are some ideas to get started:
- NCBI Gene: https://www.ncbi.nlm.nih.gov/gene/
- UniProt: https://www.uniprot.org/
- GeneCards: https://www.genecards.org/  
Also, use an AI Chatbot and compare the output to what you found in above databases.

#### 6. Simple pathway / GO term investigation
You can try to paste your lists into one of the following tools (or another one of your choosing):
- Enrichr: https://maayanlab.cloud/Enrichr/
- GOrilla https://cbl-gorilla.cs.technion.ac.il/

In [ ]:
# find the top 50 genes by padj and feed them into online gene ontology resources
